In [1]:
import nibabel
from nilearn import plotting
import os
import numpy as np
import pandas as pd
import scipy as sp
import random as rn

In [2]:
os.chdir('C:\Users\John\Documents\Code\\Python\\nsaba\\data_dir')

In [3]:
db_table = pd.read_table('database.txt')
feat_table = pd.read_table('features.txt')
expres_aba = pd.read_csv('MicroArrayExpression.csv')
annot_aba = pd.read_csv('SampleAnnot.csv')
probes_aba = pd.read_csv('Probes.csv')

In [4]:
print feat_table.shape
print db_table.shape

(10903, 3407)
(386455, 13)


In [526]:
mni_coords

array([[-10.1,   5.9,  -8.4],
       [  8.9,   8.5,  -7.4],
       [  7.9,   0.8,  -6.7],
       ..., 
       [ 26. , -15.4,  -8.8],
       [ 27.2, -15.4,  -8.8],
       [ 26. , -15.4,  -9.9]])

In [5]:
#KDTree for efficient MNI Coordinate Extraction
mni_coords = annot_aba.as_matrix()[1:, 10:].astype(float)
coord_tree = sp.spatial.KDTree(mni_coords)

pts = [rn.uniform(-10,10), rn.uniform(-10,10), rn.uniform(-10,10)]
i = coord_tree.query_ball_point(pts,8.0)
print coord_tree.data[i]

[[ 13.4   2.2   0.2]
 [  1.    3.3   4.8]
 [  2.1   4.5   7.1]
 [  3.2   8.7   3.2]]


In [6]:
# Get only MNI Coordinates
dbmni = db_table[db_table.space == 'MNI']

In [7]:
annot_aba.loc[:,'mni_x':'mni_z'].as_matrix()

array([[-29.2,   5.8,  -2.6],
       [-10.1,   5.9,  -8.4],
       [  8.9,   8.5,  -7.4],
       ..., 
       [ 26. , -15.4,  -8.8],
       [ 27.2, -15.4,  -8.8],
       [ 26. , -15.4,  -9.9]])

In [8]:
db_table = db_table.loc[db_table.space == 'MNI', ['id','x','y','z']]

In [9]:
# ID to terms
ID = 9185551
df = feat_table.loc[feat_table['pmid'] == ID, feat_table.loc[feat_table['pmid'] == ID].iloc[0] > 0]
df

,pmid,accounted,anterior,anterior temporal,bilaterally,blood,blood flow,central,cerebral,cerebral blood,...,stimulation,structures,suggest,temporal,temporal lobe,thresholds,tomography,treatment,using,water
2,9185551,0.126286,0.04888,0.112669,0.083018,0.077009,0.111414,0.081773,0.152437,0.113549,...,0.079183,0.0764,0.046337,0.179396,0.253363,0.39559,0.098813,0.092124,0.040626,0.131512


In [10]:
# Term to IDs

term = 'attention'
term_ids_act = feat_table.loc[feat_table[term] > 0, ['pmid', term]]
term_ids = term_ids_act['pmid'].tolist()
term_ids_act

,pmid,attention
7,9408106,0.059474
20,9787003,0.167045
21,9808463,0.111859
23,9886448,0.055644
39,10349031,0.090629
43,10376114,0.086412
60,10554989,0.339903
77,10648440,0.042161
89,10686176,0.109010
91,10689056,0.334503


In [12]:
# IDs to Coords
term_coords = db_table.loc[db_table['id'].isin(term_ids)]
tc = term_coords.loc[:,'x':'z'].as_matrix().astype(float)
ns_coord_tree = sp.spatial.KDTree(tc)

term_ids_act.rename(columns={'pmid':'id'}, inplace=True)
df = term_coords.merge(term_ids_act)

In [32]:
# Getting activation based on Coordinate
coords = [-241,-661,516]
%time mean = np.mean(df[(df['x'] == coords[0]) & (df['y'] == coords[1]) & (df['z'] == coords[2])][term].mean())

Wall time: 5 ms


In [ ]:
# Returns Sphere about point

def sphere():
    sphere_bucket = []
    set_bucket = []
    for i, r in enumerate(range(4,0,-1)):
        pts = ns_coord_tree.query_ball_point([-2,6,3], r)
        set_bucket.append(set(map(tuple, ns_coord_tree.data[pts])))
   
    for i in range(0,3):
        sphere_bucket.append(list(set_bucket[i].difference(set_bucket[i+1])))
    sphere_bucket.append(list(set_bucket[3]))
    rev_iter = reversed(sphere_bucket)
    
    return np.array([layer for layer in rev_iter])

def sphere()

In [ ]:
%time for i in range(800): sphere()